In [1]:
from openai import OpenAI
import os

#client.api_key = os.getenv('OPENAI_API_KEY') # asigna la API key de chatGPT

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lee el archivo .env donde está la api key


client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [2]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'MicrÃ³fono (USB PnP Sound Device', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Varios micrÃ³fonos (Realtek(R) A', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Mezcla estÃ©reo (Realtek(R) Audi', 'hostApi': 0, 'maxInputCh

In [3]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 2
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

MENSAJE = ""

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT,
                   channels=CHANNELS,
                   rate=FRAME_RATE,
                   input=True,
                   input_device_index=1,
                   frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()

In [4]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-es-0.42")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)


def speech_recognition(output):
    global MENSAJE
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        MENSAJE += text + " "
        
        output.append_stdout(text+ " ")

In [5]:
from IPython.display import Audio, display, clear_output
from pathlib import Path
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
import openai
import time
import threading
from queue import Queue
from pydub import AudioSegment
from pydub.playback import play
import pyttsx3
from docx import Document

# Initialize the conversation
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

# Load the audio file (replace 'loading_sound.mp3' with your audio file path)
loading_sound = AudioSegment.from_file('relaxing.mp3') - 20


# ... (Rest of your code remains the same)
respuesta = ""
current_audio = None



doc = Document('super_prompt.docx')

prompt = ''
for para in doc.paragraphs:
    prompt += para.text + '\n'
    
    
#print(text)


# Global flag to control the loading sound playback
stop_loading_sound = False

def play_loading_sound_continuously():
    global stop_loading_sound
    while not stop_loading_sound:
        play(loading_sound)
        time.sleep(0.05)  # Adjust the sleep time as needed


# Define a global variable to store the response text
respuesta = ""
current_audio = None
current_audio_path = None
audio_output = widgets.Output()
display(audio_output)

# Function to generate speech for the response
def generate_speech(response_text):
    global stop_loading_sound
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    
    stop_loading_sound = True

    # Selecting a Spanish voice, if available
    #for voice in voices:
    #    if 'spanish' in voice.name.lower():
    #        engine.setProperty('voice', voice.id)
    #        break
    # Get current speech rate
    current_rate = engine.getProperty('rate')
    #print(f"Current speech rate: {current_rate}")

    # Set a slower speech rate (you can adjust this value)
    new_rate = 150  # Default is usually around 200
    engine.setProperty('rate', new_rate)

    #engine.setProperty('voice', "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-ES_HELENA_11.0")
    engine.setProperty('voice', "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-MX_SABINA_11.0")        

    engine.say(response_text)
    engine.runAndWait()
    
    
    
    
respuesta = conversation.predict(input=prompt)

generate_speech(respuesta)

# ... (Rest of your code remains the same)
messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()


# Function to handle the user's speech input
def handle_user_input(user_input):
    global respuesta

    respuesta = conversation.predict(input=user_input)
    generate_speech(respuesta)

# ... (Rest of your code remains the same)

# Function to handle the "Stop" button click
def stop_recording(data):
    global current_audio_path
    global stop_loading_sound

    with output:
        messages.get()
        display("Stopped.")
        # Play the loading sound
        #play(loading_sound)
        #play(loading_sound)
        
        # Start playing the loading sound in a separate thread
        stop_loading_sound = False
        loading_thread = threading.Thread(target=play_loading_sound_continuously)
        loading_thread.start()

        #display("Transcribed Text:", MENSAJE)

        user_input = MENSAJE

        # Handle the user's input and generate a response
        handle_user_input(user_input)
        
        

        
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)


Output()

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()